# Following the low level tutorial guide
This code is currently looking at this IRIS observation:

DST flare watch coordination, 2014-10-25 14:58:28-18:00:56

OBS 3880106953: Large sit-and-stare

https://www.lmsal.com/hek/hcr?cmd=view-event&event-id=ivo%3A%2F%2Fsot.lmsal.com%2FVOEvent%23VOEvent_IRIS_20141025_145828_3880106953_2014-10-25T14%3A58%3A282014-10-25T14%3A58%3A28.xml

In [ ]:
%matplotlib inline
import gzip
import numpy as np
import astropy.io.fits as fits
import matplotlib.pyplot as plt
from astropy.wcs import WCS
from scipy.optimize import curve_fit
from scipy.stats import chisquare
import pandas as pd
from lmfit import Model
# Set up some default matplotlib options
plt.rcParams['figure.figsize'] = [15, 10]
plt.rcParams['xtick.direction'] = 'out'
plt.rcParams['image.origin'] = 'lower'
plt.rcParams['image.cmap'] = 'inferno'

## RASTER FILES

In [ ]:
# opens the IRIS fits file
sp = fits.open("20141025raster.fits")

In [ ]:
# doublecheck what type of observation we have
hd = sp[0].header
hd['OBS_DESC']

In [ ]:
# print name & index of each spectral line
print('Window. Name      : wave start - wave end\n')
for i in range(hd['NWIN']):
    win = str(i + 1)
    print('{0}. {1:15}: {2:.2f} - {3:.2f} Å'
          ''.format(win, hd['TDESC' + win], hd['TWMIN' + win], hd['TWMAX' + win]))

In [ ]:
# Set which line to look at
line_num = 2

In [ ]:
# changing this value to 1 was necessary to use the wcs transformation code (something to do with making sure the transformation matrix is non-singular)
# however I don't think we even need actual Sun coords anwyway
if sp[line_num].header['CDELT3']==0:
    sp[line_num].header['CDELT3']=1

In [ ]:
# This code is sorcery I don't understand that gets us the wavelength information
wcs = WCS(sp[line_num].header)
m_to_nm = 1e9  # convert wavelength to nm
nwave = sp[line_num].data.shape[2]
wavelength = wcs.all_pix2world(np.arange(nwave), [0.], [0.], 0)[0] * m_to_nm

# Spectroheliograms

In [ ]:
# The position of the 1393.757 Å line
line_index_1394 = np.argmin(np.abs(wavelength - 139.3757))
print(line_index_1394)

In [ ]:
# plot the brightness in the 1394 line. y axis = slit position, x axis = time
plt.figure(figsize=(20, 20))

plt.imshow(sp[line_num].data[..., line_index_1394].T, vmin=100, vmax=2000)

In [ ]:
# Let's get the flare data
temp_data = sp[line_num].data

In [ ]:
# create an array containing the timestamps of the data (as strings)
time_diff = sp[-2].data[:, sp[-2].header['TIME']]
times_temp = np.datetime64(hd['DATE_OBS']) + time_diff * np.timedelta64(1, 's')
times_string = np.datetime_as_string(times_temp)
times = []
for letter in np.arange(0, len(times_string)):
    times.append(times_string[letter][11:-4])

# Gaussian Analysis

### Gaussian Analysis Test

testing lmfit

In [ ]:
### which pixel?
test_t = 400
test_y = 299

In [ ]:
# Test on the 1394 line - we need to extract this from the full spectral data
# we use the two line_index variables found previously, but we want to be able to select a specific wavelength range around the reference wavelength

li_1394_upper = np.argmin(np.abs(wavelength - 139.42))
li_1394_lower = np.argmin(np.abs(wavelength - 139.33))

# extract the desired data range
test_gauss_x = wavelength[li_1394_lower:li_1394_upper]
test_gauss_y_32 = sp[line_num].data[test_t, test_y, li_1394_lower:li_1394_upper]
test_gauss_y = test_gauss_y_32.astype(np.float64) # conversion of y data to float64, so it matches the x data
test_y_error = np.sqrt(np.absolute(test_gauss_y)/4)+1.5

# dummy x 
dummytest = np.linspace(np.min(test_gauss_x), np.max(test_gauss_x), 1000)

# Then we can perform a Gaussian fit to the line profile
# initial guesses
mean = 139.3757
sigma = 0.01
background = 1

def gauss1(x,A1,mu1,sigma1):
    return A1*np.exp(-(x-mu1)**2/2/sigma1**2)

def gauss2(x,A2,mu2,sigma2):
    return A2*np.exp(-(x-mu2)**2/2/sigma2**2)

def backgroundFunc(x,slope,intercept):
    return slope*x + intercept

compositeModel = Model(gauss1) + Model(gauss2)# + Model(backgroundFunc)

compositeModel.set_param_hint('A1', value=0.33*max(test_gauss_y), min=0)
compositeModel.set_param_hint('mu1', value=mean, min=139.36, max=139.42)
compositeModel.set_param_hint('sigma1', value=sigma)
compositeModel.set_param_hint('A2', value=0.66*max(test_gauss_y))
compositeModel.set_param_hint('mu2', value=mean+0.01, min=139.36, max=139.42)
compositeModel.set_param_hint('sigma2', value=sigma)
#compositeModel.set_param_hint('slope', value=1, min=0.5, max=1.5)
#compositeModel.set_param_hint('intercept', value=1, max=2)
params = compositeModel.make_params()

compResult = compositeModel.fit(test_gauss_y, params, x=test_gauss_x, weights=1/test_y_error)

print(compResult.fit_report())
#result.ci_report()
#compResult.plot(yerr=test_y_error, numpoints=1000, show_init=True)

comps = compResult.eval_components()
predicted = compositeModel.eval(compResult.params, x=dummytest)
predGauss1 = Model(gauss1).eval(compResult.params, x=dummytest)
predGauss2 = Model(gauss2).eval(compResult.params, x=dummytest)

plt.errorbar(test_gauss_x,test_gauss_y, yerr=test_y_error, fmt='o', ms=4, label='data')
plt.plot(dummytest, predicted, '--', label='Total fit')
plt.plot(dummytest, predGauss1, '--', label='gauss1 component')
plt.plot(dummytest, predGauss2, '--', label='gauss2 component')
#plt.plot(test_gauss_x, comps['backgroundFunc'], '--', label='background')
plt.grid(which='major')
plt.axhline(y=0, color='b', linestyle='-')
plt.axvline(x=139.3757, color='b', linestyle='-')
plt.legend()
plt.title('Double Gaussian Fit to 1394 A profile')
plt.xlabel('Wavelength (nm)')
plt.ylabel('Intensity (DN)')
plt.show()

In [ ]:
print(compResult.params)


In [ ]:
# Test on the 1403 line - we need to extract this from the full spectral data
# we use the two line_index variables found previously, but we want to be able to select a specific wavelength range around the reference wavelength

li_1403_upper = np.argmin(np.abs(wavelength - 140.33))
li_1403_lower = np.argmin(np.abs(wavelength - 140.24))

# extract the desired data range
test_gauss_x = wavelength[li_1403_lower:li_1403_upper]
test_gauss_y_32 = sp[line_num].data[test_t, test_y, li_1403_lower:li_1403_upper]
test_gauss_y = test_gauss_y_32.astype(np.float64) # conversion of y data to float64, so it matches the x data

test_y_error = np.sqrt(np.absolute(test_gauss_y)/4)+1.5

# dummy x 
dummytest = np.linspace(np.min(test_gauss_x), np.max(test_gauss_x), 1000)

# Then we can perform a Gaussian fit to the line profile
# initial guesses
mean = 140.2772
sigma = 0.01
background = 1

def gauss3(x,A3,mu3,sigma3):
    return A3*np.exp(-(x-mu3)**2/2/sigma3**2)

compositeModel = Model(gauss1) + Model(gauss2) + Model(gauss3)

compositeModel.set_param_hint('A1', value=0.66*max(test_gauss_y), min=0)
compositeModel.set_param_hint('mu1', value=mean, min=140.26, max=140.32)
compositeModel.set_param_hint('sigma1', value=sigma)
compositeModel.set_param_hint('A2', value=0.33*max(test_gauss_y), min=0)
compositeModel.set_param_hint('mu2', value=mean+0.01, min=140.26, max=140.32)
compositeModel.set_param_hint('sigma2', value=sigma)
compositeModel.set_param_hint('A3', value=0.05*max(test_gauss_y), min=0)
compositeModel.set_param_hint('mu3', min=140.24, max=140.265, value=140.25)
compositeModel.set_param_hint('sigma3', value=sigma)
#compositeModel.set_param_hint('slope', value=1, min=0.5, max=1.5)
#compositeModel.set_param_hint('intercept', value=1, min=-2, max=2)
params = compositeModel.make_params()

compResult = compositeModel.fit(test_gauss_y, params, x=test_gauss_x, weights=1/test_y_error)

print(compResult.fit_report())
#result.ci_report()
#compResult.plot(yerr=test_y_error, numpoints=1000, show_init=True)

comps = compResult.eval_components()
predicted = compositeModel.eval(compResult.params, x=dummytest)
predGauss1 = Model(gauss1).eval(compResult.params, x=dummytest)
predGauss2 = Model(gauss2).eval(compResult.params, x=dummytest)
predGauss3 = Model(gauss3).eval(compResult.params, x=dummytest)

plt.errorbar(test_gauss_x,test_gauss_y, yerr=test_y_error, fmt='o', ms=4, label='data')
plt.plot(dummytest, predicted, '--', label='Total fit')
plt.plot(dummytest, predGauss1, '--', label='gauss1 component')
plt.plot(dummytest, predGauss2, '--', label='gauss2 component')
plt.plot(dummytest, predGauss3, '--', label='gauss3 component')
plt.grid(which='major')
plt.axhline(y=0, color='b', linestyle='-')
plt.axvline(x=140.2772, color='b', linestyle='-')
plt.legend()
plt.title('Double Gaussian Fit to 1403 A profile')
plt.xlabel('Wavelength (nm)')
plt.ylabel('Intensity (DN)')
plt.show()

## Loop Code without Chi test

In [ ]:
# create the array with 100 x 2040 data points - one for each data point in the area of interest - with columms for:
# time & slit coordinate, max intensity for both lines, a flag for if lines are single Gaussian, and all Gauss info
# the y-coordinate will be zero-based from y = 288 on the spectroheliograph
tripleGaussArray = np.zeros((2040, 85, 21))

# line ranges
li_1394_upper = np.argmin(np.abs(wavelength - 139.42))
li_1394_lower = np.argmin(np.abs(wavelength - 139.33))

li_1403_upper = np.argmin(np.abs(wavelength - 140.33))
li_1403_lower = np.argmin(np.abs(wavelength - 140.24))

# x data
x1394 = wavelength[li_1394_lower:li_1394_upper]
dummyx1394 = np.linspace(np.min(x1394), np.max(x1394), 1000)
x1403 = wavelength[li_1403_lower:li_1403_upper] 
dummyx1403 = np.linspace(np.min(x1403), np.max(x1403), 1000)

# slit positions used
startslit = 288
endslit = 373
#endslit = 289

# construct composite models
def gauss1(x,A1,mu1,sigma1):
    return A1*np.exp(-(x-mu1)**2/2/sigma1**2)

def gauss2(x,A2,mu2,sigma2):
    return A2*np.exp(-(x-mu2)**2/2/sigma2**2)

def gauss3(x,A3,mu3,sigma3):
    return A3*np.exp(-(x-mu3)**2/2/sigma3**2)

def backgroundFunc(x,slope,intercept):
    return slope*x + intercept

doubleGaussModel = Model(gauss1) + Model(gauss2)# + Model(backgroundFunc)
tripleGaussModel = Model(gauss1) + Model(gauss2) + Model(gauss3)

# initial guesses
mean_1394 = 139.3757
mean_1403 = 140.2772
sigma = 0.01
background = 1

gny = 0
success_1394 = 0
success_1403 = 0
for gy in np.arange(startslit, endslit):
    print(gy)
    for gt in np.arange(0, 2040):
        # store coords of this point
        tripleGaussArray[gt, gny, 0] = gt
        tripleGaussArray[gt, gny, 1] = gy
        
        # extract the desired data range
        gauss_y_32 = sp[line_num].data[gt, gy, li_1394_lower:li_1394_upper]
        gauss_y = gauss_y_32.astype(np.float64) # conversion of y data to float64, so it matches the x data
        yError = np.sqrt(np.absolute(gauss_y)/4)+1.5
        #store intensity at this point
        tripleGaussArray[gt, gny, 2] = max(gauss_y)
        
        doubleGaussModel.set_param_hint('A1', value=0.33*max(gauss_y), min=0)
        doubleGaussModel.set_param_hint('mu1', value=mean_1394, min=139.36, max=139.42)
        doubleGaussModel.set_param_hint('sigma1', value=sigma)
        doubleGaussModel.set_param_hint('A2', value=0.66*max(gauss_y), min=0)
        doubleGaussModel.set_param_hint('mu2', value=mean_1394+0.01, min=139.36, max=139.42)
        doubleGaussModel.set_param_hint('sigma2', value=sigma)
        #doubleGaussModel.set_param_hint('slope', value=1, min=0.5, max=1.5)
        #doubleGaussModel.set_param_hint('intercept', value=1, max=2)
        paramsDouble = doubleGaussModel.make_params()
        
        # perform the Gaussian fit
        try:
            
            doubleResult = doubleGaussModel.fit(gauss_y, paramsDouble, x=x1394, weights=1/yError)

            tripleGaussArray[gt, gny, 5] = np.max(doubleGaussModel.eval(doubleResult.params, x=dummyx1394))
            tripleGaussArray[gt, gny, 6] = dummyx1394[np.argmax(doubleGaussModel.eval(doubleResult.params, x=dummyx1394))]
            tripleGaussArray[gt, gny, 7] = np.max(Model(gauss1).eval(doubleResult.params, x=dummyx1394))
            tripleGaussArray[gt, gny, 8] = dummyx1394[np.argmax(Model(gauss1).eval(doubleResult.params, x=dummyx1394))]
            tripleGaussArray[gt, gny, 9] = np.max(Model(gauss2).eval(doubleResult.params, x=dummyx1394))
            tripleGaussArray[gt, gny, 10] = dummyx1394[np.argmax(Model(gauss2).eval(doubleResult.params, x=dummyx1394))]
            #tripleGaussArray[gt, gny, 19] = popt[6]
            # chi square 5% level = 43.773, 1% = 50.892 (for dof = 30)
            if doubleResult.redchi < 43.773:
                success_1394 = 1
            else:
                success_1394 = 0
        except:
            success_1394 = 2
        
        if success_1394 == 1:
            #do the fitting for the 1403 line
            gauss_y_32 = sp[line_num].data[gt, gy, li_1403_lower:li_1403_upper]
            gauss_y = gauss_y_32.astype(np.float64) # conversion of y data to float64, so it matches the x data
            yError = np.sqrt(np.absolute(gauss_y)/4)+1.5
            #store intensity at this point
            tripleGaussArray[gt, gny, 3] = max(gauss_y)

            tripleGaussModel.set_param_hint('A1', value=0.66*max(gauss_y), min=0)
            tripleGaussModel.set_param_hint('mu1', value=mean_1403, min=140.26, max=140.32)
            tripleGaussModel.set_param_hint('sigma1', value=sigma)
            tripleGaussModel.set_param_hint('A2', value=0.33*max(gauss_y), min=0)
            tripleGaussModel.set_param_hint('mu2', value=mean_1403+0.01, min=140.26, max=140.32)
            tripleGaussModel.set_param_hint('sigma2', value=sigma)
            tripleGaussModel.set_param_hint('A3', value=0.05*max(gauss_y), min=0)
            tripleGaussModel.set_param_hint('mu3', min=140.24, max=140.265, value=140.25)
            tripleGaussModel.set_param_hint('sigma3', value=sigma)
            paramsTriple = tripleGaussModel.make_params()
            # perform the Gaussian fit
            try:
                
                tripleResult = tripleGaussModel.fit(gauss_y, paramsTriple, x=x1403, weights=1/yError)

                tripleGaussArray[gt, gny, 11] = np.max(tripleGaussModel.eval(tripleResult.params, x=dummyx1403))
                tripleGaussArray[gt, gny, 12] = dummyx1403[np.argmax(tripleGaussModel.eval(tripleResult.params, x=dummyx1403))]
                tripleGaussArray[gt, gny, 13] = np.max(Model(gauss1).eval(tripleResult.params, x=dummyx1403))
                tripleGaussArray[gt, gny, 14] = dummyx1403[np.argmax(Model(gauss1).eval(tripleResult.params, x=dummyx1403))]
                tripleGaussArray[gt, gny, 15] = np.max(Model(gauss2).eval(tripleResult.params, x=dummyx1403))
                tripleGaussArray[gt, gny, 16] = dummyx1403[np.argmax(Model(gauss2).eval(tripleResult.params, x=dummyx1403))]
                tripleGaussArray[gt, gny, 17] = np.max(Model(gauss3).eval(tripleResult.params, x=dummyx1403))
                tripleGaussArray[gt, gny, 18] = dummyx1403[np.argmax(Model(gauss3).eval(tripleResult.params, x=dummyx1403))]
                #tripleGaussArray[gt, gny, 20] = popt[9]

                if tripleResult.redchi < 43.773:
                    success_1403 = 1
                else:
                    success_1403 = 0
            except:
                success_1403 = 2

        # did the gaussian fitting succeed?
        if success_1394==1 and success_1403==1:
            tripleGaussArray[gt,gny, 4] = 1
        else:
            tripleGaussArray[gt,gny, 4] = 0
        
        success_1394 = 0
        success_1403 = 0
    gny = gny + 1

In [ ]:
no_of_data=np.count_nonzero(tripleGaussArray[:,:,4] == 1, axis=0)
print(no_of_data)
print('We currently have ', np.sum(no_of_data), ' data points where both profiles are double Gaussian')
resultsArray = tripleGaussArray.reshape(2040*85,21)

In [ ]:
#save data as csv by converting to pandas df
colnames = ['t coord', 'y coord', '1394 int', '1403 int', 'success', '1394g int', '1394g wave', '1394g1 int', '1394g1 wave', '1394g2 int', '1394g2 wave',
            '1403g int', '1403g wave', '1403g1 int', '1403g1 wave', '1403g2 int', '1403g2 wave', '1403g3 int', '1403g3 wave', '1394back', '1403back']
dfgauss = pd.DataFrame(resultsArray, columns=colnames)
dfgauss.to_csv('lmfitData.csv', index=False)